### Esercizio 1: 
Hai a disposizione un file `data.csv` contenente dati mensili di passeggeri con due colonne:

- `date`: data in formato `YYYY-MM` (mese/anno)
- `passengers`: numero di passeggeri per quel mese


Costruisci un modello di **regressione polinomiale** che approssima l’andamento del numero di passeggeri nel tempo.

1. Carica il dataset.
2. Convertilo in un formato numerico utilizzando una colonna `mese_numerico` che conti i mesi a partire da gennaio 1949.
3. Applica una regressione polinomiale (grado a tua scelta).
4. Calcola l’RMSE tra i valori reali e quelli predetti.
5. Visualizza i dati reali e la curva stimata con Plotly.

***Punto 1: Carico il dataset***

In [6]:
import pandas as pd

# Importo il file CSV
df = pd.read_csv ("data.csv")

# Visualizzo le prime 5 righe per controllare la struttura
df.head()

,date,passengers
0,1949-01,112.0
1,1949-02,118.0
2,1949-03,132.0
3,1949-04,129.0
4,1949-05,121.0


***Punto 2: Aggiunta della colonna mese_numerico***


In [7]:
# Converto la colonna 'date' in oggetti datetime
df['date'] = pd.to_datetime(df['date'])

# Calcolo i mesi trascorsi da gennaio 1949
df['mese_numerico'] = (df['date'].dt.year - 1949) * 12 + (df['date'].dt.month - 1)

# Controllo le prime righe per vedere se la colonna è stata aggiunta correttamente
df.head()

## INFO: 
#   - pd.to_datetime(...): Converte le stringhe di data in oggetti datetime.
#   - (year - 1949) * ...: Calcola il numero di mesi trascorsi da Gennaio 1949.

# Il mese 0 = gennaio 1949, mese 1 = febbraio 1949, ecc..

,date,passengers,mese_numerico
0,1949-01-01,112.0,0
1,1949-02-01,118.0,1
2,1949-03-01,132.0,2
3,1949-04-01,129.0,3
4,1949-05-01,121.0,4


***Punto 3: Regressione polinomiale***

In [8]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

# Converto la colonna passengers in numerico 
df['passengers'] = pd.to_numeric(df['passengers'], errors='coerce')

# Rimuovo eventuali righe con valori mancanti
df_clean = df.dropna()

# Separo le variabili
X = df_clean[['mese_numerico']]  # variabile indipendente (X)
y = df_clean['passengers']       # variabile dipendente (y)

# Trasformo X in variabili polinomiali fino al grado 2
poly = PolynomialFeatures(degree=2)
X_poly = poly.fit_transform(X)

# Creo e adatto il modello
model = LinearRegression()
model.fit(X_poly, y)

# Calcolo le predizioni sui dati originali
y_pred = model.predict(X_poly)

## INFO: 
#   - PolynomialFeatures(...): Crea nuove colonne con i termini quadratici;
#   - LinearRegression().fit(...): Costruisce il modello sui dati trasformati.
#   - y_pred: Sono i valori predetti dal modello.

***Punto 4: Calcolo RMSE (Root Mean Squared Error)***

In [9]:
from sklearn.metrics import mean_squared_error
import numpy as np

# Calcolo l'errore quadratico medio (MSE)
mse = mean_squared_error(y, y_pred)

# Calcolo la radice quadrata per ottenere l'RMSE
rmse = np.sqrt(mse)

# Stampo il risultato
print("RMSE:", rmse)

## INFO:
#   - mean_squared_error: Calcola la media degli errori quadratici tra y e y_pred.
#   - np.sqrt(...): Trasforma l’MSE in RMSE (che ha le stesse unità dei passeggeri).

# Più il valore è vicino a 0, più il modello è preciso.

RMSE: 44.45027656808772


***Punto 5: Visualizza i dati reali e la curva stimata con Plotly***

In [20]:
import plotly.graph_objects as go
import plotly.io as pio

# Imposto il renderer per VS Code
pio.renderers.default = "vscode"

# Creo il grafico interattivo
fig = go.Figure()

# Punti reali dei passeggeri
fig.add_trace(go.Scatter(
    x=df_clean['mese_numerico'],        # Ascissa: mesi del gennaio 1949
    y=y,                                # Ordinata: Valori reali dei passeggeri
    mode='markers',
    name='Dati reali'
))

# Curva stimata dal modello
fig.add_trace(go.Scatter(
    x=df_clean['mese_numerico'],       # Stessa ascissa
    y=y_pred,                          # Ordinata: Valori stimati dal modello polinomiale
    mode='lines',
    name='Regressione polinomiale'
))

# Titoli e assi
fig.update_layout(
    title='Numero di passeggeri nel tempo',
    xaxis_title='Mese (da Gennaio 1949)',
    yaxis_title='Numero passeggeri'
)

# Mostro il grafico
fig.show()

## INFO: 
#   - go.Scatter(..., mode='markers'): Mostra i dati reali come punti.
#   - go.Scatter(..., mode='lines'): Mostra la curva stimata.
#   - fig.show(): Apre il grafico interattivo.


### Esercizio 2: 
Costruisci una web app con Dash che permette all’utente di scegliere il grado del polinomio per adattare un modello di regressione ai dati non lineari e vedere il risultato aggiornarsi dinamicamente.


1. Genera 100 punti x tra -3 e 3.

2. Calcola ad esempio y = x³ - x + rumore.

3. Costruisci un'interfaccia Dash con:
    - uno slider per scegliere il grado del polinomio (1–10),
    - un grafico Plotly che mostra i dati e la curva stimata.

4. Usa PolynomialFeatures + LinearRegression da scikit-learn per stimare la curva

***Punto 1: Genera i dati***

In [ ]:
import numpy as np
import pandas as pd

# Genera 100 punti x equispaziati tra -3 e 3
x = np.linspace(-3, 3, 100)

# Calcola y = x³ - x + rumore gaussiano
np.random.seed(0)  # per risultati ripetibili
rumore = np.random.normal(0, 1, size=x.shape)
y = x**3 - x + rumore

# Creo un DataFrame per comodità
df = pd.DataFrame({'x': x, 'y': y})

# Visualizzo le prime righe
df.head()

## INFO:
#   -np.linspace(...): Crea 100 valori equidistanti tra -3 e 3.
#   -np.random.normal(...): Aggiunge rumore gaussiano.
#   -y = x³ - x + rumore: Funzione target non lineare.


,x,y
0,-3.000000,-22.235948
1,-2.939394,-22.056920
2,-2.878788,-20.000197
3,-2.818182,-17.323344
4,-2.757576,-16.344090


***Punto 2: Costruzione dell’app Dash di base***

In [24]:
from dash import Dash, dcc, html
import plotly.graph_objects as go

# Creo l'app Dash
app = Dash(__name__)

# Layout dell'app: slider + grafico
app.layout = html.Div([
    html.H1("Regressione Polinomiale Interattiva"),
    
    # Slider per scegliere il grado (1–10)
    dcc.Slider(
        id='grado-slider',
        min=1,
        max=10,
        step=1,
        value=2,  # valore iniziale
        marks={i: str(i) for i in range(1, 11)}
    ),
    
    # Grafico dove mostreremo dati + curva stimata
    dcc.Graph(id='grafico-output')
])

## INFO: 
#   - html.H1(...): Titolo principale.
#   -dcc.Slider(...): Consente di scegliere il grado del polinomio (da 1 a 10).
#   -dcc.Graph(...): Componente che ospiterà il grafico Plotly.

***Punto3: Callback dinamico + regressione polinomiale***

In [ ]:
from dash.dependencies import Input, Output
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

@app.callback(
    Output('grafico-output', 'figure'),
    Input('grado-slider', 'value')
)
def aggiorna_grafico(grado):
    # Preparo i dati di input
    X = df[['x']]
    y_true = df['y']
    
    # Trasformo X in polinomi del grado selezionato
    poly = PolynomialFeatures(degree=grado)
    X_poly = poly.fit_transform(X)
    
    # Fitting del modello
    model = LinearRegression()
    model.fit(X_poly, y_true)
    
    # Predizione
    y_pred = model.predict(X_poly)

    # Creo il grafico Plotly
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df['x'], y=y_true, mode='markers', name='Dati reali'))
    fig.add_trace(go.Scatter(x=df['x'], y=y_pred, mode='lines', name=f'Polinomio grado {grado}'))

    fig.update_layout(title='Regressione polinomiale dinamica',
                      xaxis_title='x',
                      yaxis_title='y')

    return fig

## INFO: 
#   -@app.callback(...): Crea un legame automatico tra slider e grafico.
#   -PolynomialFeatures(...) + LinearRegression(...): Fanno il fit in tempo reale.

#   Ogni volta che grado-slider cambia, la funzione aggiorna_grafico viene chiamata.


***Punto 4: Avvio dell'app***

In [27]:
if __name__ == '__main__':
    app.run(debug=True)

## INFO: 
#   -if __name__ == '__main__': Serve per eseguire l'app solo quando il file è lanciato direttamente (non importato).
#   -app.run(debug=True): Avvia il server in locale e ti permette di vedere l’app sul browser all’indirizzo: